In [49]:
from requests import Request, Session
import pandas as pd
import json
import time
import webbrowser
import pprint
import gspread
import csv
from datetime import date, timedelta
from dateutil import parser

headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': '5af15786-87c3-45f8-92de-214a92592503'
}

session = Session()
session.headers.update(headers)

today = date.today()
d_min = date.today() - timedelta(days=365)

## GET TOP 10 EXCHANGE LISTS

In [50]:
url_map_id = 'https://pro-api.coinmarketcap.com/v1/exchange/map'
parameters_map_id = { 'start' : 1,
               'limit' : 10,
               'sort' : 'volume_24h',
               }
response_map_id = session.get(url_map_id, params = parameters_map_id)
top_xc = pd.json_normalize(json.loads(response_map_id.text)['data'])
top_xc = top_xc[['id','name','slug']]
top_xc

,id,name,slug
0,270,Binance,binance
1,1563,BTCEX,btcex-exchange
2,294,OKX,okx
3,544,MEXC,mxc
4,521,Bybit,bybit
5,1064,BingX,bingx
6,1182,Deepcoin,deepcoin
7,524,FTX,ftx
8,525,XT.COM,xt
9,513,Bitget,bitget


url = 'https://pro-api.coinmarketcap.com/v1/exchange/market-pairs/latest'

c = pd.DataFrame()
for i in top_xc['slug']:
    parameters = { 'slug' : i,
               'start' : 1,
               'limit' : 10,
               'category' : 'spot'
               }
    response = session.get(url, params=parameters)
    top_xc1 = pd.json_normalize(json.loads(response.text))
    c = pd.concat([c,top_xc1])

c

## GET EXCHANGE VOLUME BY DATE

In [39]:
url_historical = 'https://pro-api.coinmarketcap.com/v1/exchange/quotes/historical'

list_xc = list(top_xc['slug'])
slug_xc = ','.join(list_xc)
parameters_historical = { 'slug': slug_xc,
                          'interval' : 'daily',
                          'time_start' : d_min,
                          'time_end' : today}

response_historical = session.get(url_historical, params = parameters_historical)
df = pd.DataFrame()

for i in list_xc:
    a = pd.json_normalize(json.loads(response_historical.text)['data'][i]['quotes'])
    a['exchange'] = i
    a = a.rename(columns= {'quote.USD.volume_24h':'usd_vol24h'})
    a = a[['timestamp',
           'exchange',
           'usd_vol24h']]
    df = pd.concat([df,a])

df

,timestamp,exchange,usd_vol24h
0,2021-09-28T00:00:13.000Z,binance,7.236399e+10
1,2021-09-29T00:00:16.000Z,binance,7.288293e+10
2,2021-09-30T00:00:13.000Z,binance,6.753848e+10
3,2021-10-01T00:00:15.000Z,binance,7.388184e+10
4,2021-10-02T00:00:18.000Z,binance,1.040160e+11
...,...,...,...
355,2022-09-18T00:04:11.000Z,bitget,4.744124e+09
356,2022-09-19T00:04:11.000Z,bitget,5.886741e+09
357,2022-09-20T00:04:11.000Z,bitget,8.292559e+09
358,2022-09-21T00:04:11.000Z,bitget,6.931889e+09


## MANUAL INPUT EXCHANGE LISTS

url_historical = 'https://pro-api.coinmarketcap.com/v1/exchange/quotes/historical'

list_xc = 'binance,ftx,kucoin,huobi-global,bybit,coinbase-exchange,tokocrypto,indodax,rekeningku-com'
list_xc = list_xc.split(',')
d_min = date.today() - timedelta(days=330)
slug_xc = ','.join(list_xc)
parameters_historical = { 'slug': slug_xc,
                          'interval' : 'monthly',
                          'time_start' : d_min,
                          'time_end' : today}

response_historical = session.get(url_historical, params = parameters_historical)
df = pd.DataFrame()

for i in list_xc:
    a = pd.json_normalize(json.loads(response_historical.text)['data'][i]['quotes'])
    a['exchange'] = i
    a = a.rename(columns= {'quote.USD.volume_24h':'usd_vol24h'})
    a = a[['timestamp',
           'exchange',
           'usd_vol24h']]
    df = pd.concat([df,a])

df

In [60]:
df.to_csv('/Users/ghazian/Downloads/historical_volume.csv')